In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools


from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, AvgPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau


Using TensorFlow backend.


In [0]:
# import warnings
# warnings.filterwarnings('ignore')

## Load Data , test and train

In [0]:

train = pd.read_csv("train.csv")
test = pd.read_csv("../test.csv")

In [0]:

Y_train = train["label"]
print(Y_train.shape)

X_train = train.drop(labels = ["label"],axis = 1) 
print(X_train.shape)


(70000,)
(70000, 784)


In [0]:

X_train = X_train / 255.0
test = test / 255.0

In [0]:

X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

In [0]:

Y_train = to_categorical(Y_train, num_classes = 10)

In [0]:

random_seed = 46

# Test Size Small cause wanna train with all data we have.
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.001, random_state=random_seed)

In [0]:
np.shape(X_train)

(69860, 28, 28, 1)

## Model Construction : 

In [0]:

input_shape = (28,28,1)
def Build_Model(): 

    model = Sequential()
    # 1 conv Layer
    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',  activation ='relu', input_shape = input_shape))
    model.add(BatchNormalization())
    # 2 conv Layer
    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'))
    model.add(BatchNormalization())
    model.add(AvgPool2D(pool_size=(2,2)))
    # 3 #  conv Layer
    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
    model.add(BatchNormalization())
    # 4 # First conv Layer
    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',  activation ='relu'))
    model.add(BatchNormalization())
    model.add(AvgPool2D(pool_size=(2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(256*6, activation = "relu"))
    model.add(Dropout(0.25))
    model.add(Dense(256*3, activation = "relu"))
    model.add(Dropout(0.25))
    model.add(Dense(256, activation = "relu"))
    model.add(Dropout(0.25))
    model.add(Dense(10, activation = "softmax"))

    # Compile model
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])



    return model




## Data augmentation : 

In [0]:
# data augmentation 
datagen = ImageDataGenerator(
        featurewise_center=False,  			
        samplewise_center=False,  				
        featurewise_std_normalization=False,    
        samplewise_std_normalization=False,  	
        zca_whitening=False,  					
        rotation_range=10,  					
        zoom_range = 0.1, 						
        width_shift_range=0.1,  				
        height_shift_range=0.1,  				
        horizontal_flip=False,  				
        vertical_flip=False)  					


datagen.fit(X_train)

## Traing off the model : 

In [0]:
model = Build_Model()


learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

In [0]:
# Train
batch_size = 90
history = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size), epochs = 150, validation_data = (X_val,Y_val),steps_per_epoch=X_train.shape[0] // batch_size, callbacks=[learning_rate_reduction])

Epoch 1/150
777/777 [==============================] - 33s 43ms/step - loss: 0.1871 - accuracy: 0.9450 - val_loss: 0.0234 - val_accuracy: 1.0000
Epoch 2/150
  5/777 [..............................] - ETA: 26s - loss: 0.1462 - accuracy: 0.9711

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


777/777 [==============================] - 26s 33ms/step - loss: 0.0830 - accuracy: 0.9781 - val_loss: 0.0350 - val_accuracy: 0.9857
Epoch 3/150
777/777 [==============================] - 27s 34ms/step - loss: 0.0652 - accuracy: 0.9832 - val_loss: 0.0267 - val_accuracy: 0.9857
Epoch 4/150
777/777 [==============================] - 26s 34ms/step - loss: 0.0559 - accuracy: 0.9856 - val_loss: 0.0084 - val_accuracy: 1.0000
Epoch 5/150
777/777 [==============================] - 26s 34ms/step - loss: 0.0518 - accuracy: 0.9872 - val_loss: 0.0162 - val_accuracy: 1.0000
Epoch 6/150
777/777 [==============================] - 26s 34ms/step - loss: 0.0485 - accuracy: 0.9879 - val_loss: 0.0202 - val_accuracy: 0.9857
Epoch 7/150
777/777 [==============================] - 26s 34ms/step - loss: 0.0447 - accuracy: 0.9888 - val_loss: 0.0208 - val_accuracy: 1.0000
Epoch 8/150
777/777 [==============================] - 26s 34ms/step - loss: 0.0447 - accuracy: 0.9894 - val_loss: 0.0148 - val_accuracy: 1.00

In [2]:
# serialize model to JSON
i=0
model_json = model.to_json()
with open("digits_avgPool_150_epochs.json", "w") as json_file:
      json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("digits_avgPool_150_epochs.h5")
print(" model saved to disk")


 model saved to disk


In [3]:
"""
plt.figure(figsize=(20,5))
plt.subplot(1,2,1 )
plt.plot(history.history['loss'], color='b', label="Training loss")
plt.plot(history.history['val_loss'], color='r', label="validation loss")
plt.legend(loc='best', shadow=True)

plt.subplot(1,2,2 )
plt.plot(history.history['accuracy'], color='b', label="Training accuracy")
plt.plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
plt.legend(loc='best', shadow=True)
"""

'\nplt.figure(figsize=(20,5))\nplt.subplot(1,2,1 )\nplt.plot(history.history[\'loss\'], color=\'b\', label="Training loss")\nplt.plot(history.history[\'val_loss\'], color=\'r\', label="validation loss")\nplt.legend(loc=\'best\', shadow=True)\n\nplt.subplot(1,2,2 )\nplt.plot(history.history[\'accuracy\'], color=\'b\', label="Training accuracy")\nplt.plot(history.history[\'val_accuracy\'], color=\'r\',label="Validation accuracy")\nplt.legend(loc=\'best\', shadow=True)\n'

## prediction and save prediction file

In [0]:
# 
results = model.predict(test)

results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Label")

In [0]:
# file to save and submit to kaggle.
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("Predictin_avgPooling_150Epochs.csv",index=False)